In [2]:
import pandas as pd
import plotly.express as px
import dash
import dash_bootstrap_components as dbc
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import webbrowser
import datetime

#=============================================================================
# DATA PREPARATION
#=============================================================================
df = pd.read_excel("HRDataset_Huy Nguyen.xlsx")

gender_map = {0: "Female", 1: "Male"}
employment_map = {0: "Employed", 1: "Terminated"}
df["Gender"] = df["GenderID"].map(gender_map)
df["Employment Status"] = df["Termd"].map(employment_map)
df['Year'] = pd.to_datetime(df['LastPerformanceReview_Date']).dt.year

current_year = datetime.datetime.now().year
df["Age"] = current_year - pd.to_datetime(df["DOB"]).dt.year
age_bins = [18, 25, 35, 45, 55, 100]
age_labels = ["18-25", "26-35", "36-45", "46-55", "56+"]
df["Age Group"] = pd.cut(df["Age"], bins=age_bins, labels=age_labels)

departments = ['All'] + df['Department'].unique().tolist()

#=============================================================================
# APP INITIALIZATION
#=============================================================================

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

dept_opts = [{'label': "All", 'value': "All"}] 
dept_opts += [{'label': d, 'value': d} for d in df['Department'].dropna().unique()]

pos_opts = [{'label': "All", 'value': "All"}]
pos_opts += [{'label': p, 'value': p} for p in df['Position'].dropna().unique()]

#=============================================================================
# SHARED STYLES
#=============================================================================

COLORS = {
    'primary': '#2171B5',    # Deep Blue
    'secondary': '#A6CEE3',  # Light Blue
    'background': '#E6F2FF', # Very Light Blue
    'text': '#023858',       # Dark Blue
    'white': 'white'
}

def create_card(children, className="mb-3"):
    return dbc.Card(
        dbc.CardBody(children),
        className=className
    )

#=============================================================================
# DROPDOWN COMPONENTS
#=============================================================================
dropdowns = html.Div([
    html.H5("Filter Employees", className="text-center mb-2"),
    dbc.Row([
        dbc.Col([
            html.Label("Select Department", className="mb-1"),
            dcc.Dropdown(id='department-filter', options=dept_opts, value="All", clearable=False)
        ], width=6),
        dbc.Col([
            html.Label("Select Position", className="mb-1"),
            dcc.Dropdown(id='position-filter', options=pos_opts, value="All", clearable=False)
        ], width=6),
    ], className="mb-4")
])

#=============================================================================
# ORGANIZATION INSIGHTS LAYOUT
#=============================================================================
summary_layout = html.Div([
    dropdowns,

    dbc.Row([
        dbc.Col(create_card([
            html.H5("Current Employees"),
            html.Div(id='current-employees', className="info-box")
        ]), width=3),

        dbc.Col(create_card([
            html.H5("Avg Engagement Score"),
            html.Div(id='avg-engagement', className="info-box")
        ]), width=3),

        dbc.Col(create_card([
            html.H5("Avg Satisfaction Score"),
            html.Div(id='avg-satisfaction', className="info-box")
        ]), width=3),

        dbc.Col(create_card([
            html.H5("Avg Age"),
            html.Div(id='avg-age', className="info-box")
        ]), width=3)
    ], className="mb-3"),
    
    dbc.Row([
        dbc.Col(create_card([
            dcc.Graph(id='employees-by-race')
        ]), width=6),
        
        dbc.Col(create_card([
            dcc.Graph(id='employees-by-state')
        ]), width=6),
    ])
], className="p-4 rounded", style={'backgroundColor': COLORS['background'], 'height': '90vh',
      'display': 'flex', 'flexDirection': 'column', 'justifyContent': 'start'})

#=============================================================================
# EMPLOYEE INSIGHTS LAYOUT
#=============================================================================
demographics_layout = html.Div([
    html.Div(id='button-container', children=[
        dbc.Button("Male", id="male-btn", outline=True, color="primary", size="lg", className="mx-2"),
        dbc.Button("Female", id="female-btn", outline=True, color="danger", size="lg", className="mx-2"),
    ], className="text-center mb-3"),

    dbc.Row([
        dbc.Col(create_card([
            dcc.Graph(id='race-pie-chart')
        ]), width=6),
        
        dbc.Col(create_card([
            dcc.Graph(id='age-employment-bar')
        ]), width=6),
    ], className="mb-3"),
    
    dbc.Row([
        dbc.Col(create_card([
            html.H3("Absences Analysis by Department and Gender", className="text-center mb-2"),
            
            dbc.Row([
                dbc.Col([
                    html.H5("Department"),
                    dcc.Dropdown(
                        id='dropdown_department',
                        options=[{'label': dept, 'value': dept} for dept in departments],
                        value='All',
                        clearable=False
                    )
                ], width=6),
                
                dbc.Col([
                    html.H5("Absences Range"),
                    dcc.Slider(
                        min=df['Absences'].min(), 
                        max=df['Absences'].max(), 
                        step=1,
                        value=df['Absences'].median(),
                        id='slider_absences',
                        marks={i: str(i) for i in range(
                            int(df['Absences'].min()), 
                            int(df['Absences'].max()) + 1
                        )}
                    )
                ], width=6),
            ], className="mb-2"),
            
            dcc.Graph(id='figure_salary_performance', style={'height': '300px'})
        ]), width=12, className="d-flex justify-content-center")
    ])
], className="p-4 rounded", style={'backgroundColor': COLORS['background'], 'overflowY': 'auto', 'height': '90vh'})


#=============================================================================
# SALARY INSIGHTS LAYOUT
#=============================================================================
perf_options = [{'label': "All", 'value': "All"}]
perf_options += [{'label': score, 'value': score} for score in df['PerformanceScore'].dropna().unique()]

salary_layout = html.Div([
    html.Div(id='selected-year', style={'display': 'none'}, children=str(df['Year'].max())),
    
    dbc.Row([
        dbc.Col(create_card([
            dcc.Graph(id='unfiltered_salary_graph')
        ]), width=6),
        
        dbc.Col(create_card([
            html.Label("Filter by Performance Score", className="mb-2"),
            dcc.Dropdown(
                id='performance_filter',
                options=perf_options,
                value="All",
                clearable=False,
                placeholder="Select Performance Score"
            ),
            dcc.Graph(id='avg_salary_by_performance')
        ]), width=6)
    ])
], className="p-4 rounded", style={'backgroundColor': COLORS['background']})

#=============================================================================
# MAIN APP LAYOUT
#=============================================================================

nav_pills = dbc.Nav([
    dbc.NavItem(dbc.NavLink("Organization Insights", id="tab-1", href="/", active=True, 
                            className="mb-2 text-center", style={'width': '200px', 'color': COLORS['text']})),
    dbc.NavItem(dbc.NavLink("Employee Insights", id="tab-2", href="/demographics", active=False, 
                            className="mb-2 text-center", style={'width': '200px', 'color': COLORS['text']})),
    dbc.NavItem(dbc.NavLink("Salary Insights", id="tab-3", href="/salary", active=False, 
                            className="mb-2 text-center", style={'width': '200px', 'color': COLORS['text']})),
], vertical=True, pills=True, id="sidebar-nav", className="flex-column align-items-center")


app.layout = dbc.Container([
    dbc.Row([
        # Sidebar column
        dbc.Col([
            html.H2("HR Dashboard", className="text-center mb-4", style={'color': COLORS['primary']}),
            nav_pills
        ], width=2, className="d-flex flex-column align-items-center py-4", 
           style={'backgroundColor': COLORS['secondary'], 'height': '100vh'}),

        # Main content column
        dbc.Col([
            dcc.Location(id='url', refresh=False),
            html.Div(id='page-content', className="bg-white p-3 rounded")
        ], width=10, className="bg-white p-2 rounded")
    ])
], fluid=True)

#=============================================================================
# POSITION FILTER CALLBACK
#=============================================================================
@app.callback(
    Output('position-filter', 'options'),
    Input('department-filter', 'value')
)
def update_positions(department):
    if department == "All":
        positions = df['Position'].dropna().unique()
        return [{'label': "All", 'value': "All"}] + [{'label': pos, 'value': pos} for pos in positions]
    
    filtered = df[df['Department'] == department]['Position'].dropna().unique()
    return [{'label': "All", 'value': "All"}] + [{'label': pos, 'value': pos} for pos in filtered]

#=============================================================================
# SUMMARY METRICS CALLBACK
#=============================================================================
@app.callback(
    [Output('current-employees', 'children'),
     Output('avg-engagement', 'children'),
     Output('avg-satisfaction', 'children'),
     Output('avg-age', 'children')],
    [Input('department-filter', 'value'), Input('position-filter', 'value')]
)
def update_summary(department, position):
    filtered = df[df['Termd'] == 0]
    
    if department != "All":
        filtered = filtered[filtered['Department'] == department]
    if position != "All":
        filtered = filtered[filtered['Position'] == position]
    
    emp_count = len(filtered)
    eng_score = round(filtered['EngagementSurvey'].mean(), 2)
    sat_score = round(filtered['EmpSatisfaction'].mean(), 2)
    avg_age = round(filtered['Age'].mean(), 2)
    
    return emp_count, eng_score, sat_score, avg_age

#=============================================================================
# SUMMARY GRAPHS CALLBACK
#=============================================================================
@app.callback(
    [Output('employees-by-race', 'figure'),
     Output('employees-by-state', 'figure')],
    [Input('department-filter', 'value'), Input('position-filter', 'value')]
)
def update_graphs(department, position):
    filtered = df[df['Termd'] == 0]
    
    if department != "All":
        filtered = filtered[filtered['Department'] == department]
    if position != "All":
        filtered = filtered[filtered['Position'] == position]
    
    race_df = filtered.groupby("RaceDesc").size().reset_index(name="Count")
    race_df = race_df.rename(columns={"RaceDesc": "Race"})
    race_df = race_df.sort_values("Count", ascending=True)
    
    state_df = filtered.groupby("State").size().reset_index(name="Count")
    state_df = state_df.sort_values("Count", ascending=True).tail(5)
    
    # Gradient blue color scheme
    race_max = race_df["Count"].max() if not race_df.empty else 1
    race_df["Opacity"] = 0.4 + (race_df["Count"] / race_max) * 0.6
    race_colors = [f'rgba(41, 128, 185, {opacity})' for opacity in race_df["Opacity"]]
    
    race_fig = px.bar(
        race_df, 
        y="Race", 
        x="Count", 
        orientation='h', 
        title="Racial Demographic Breakdown in Current Workforce", 
        text="Count",
        template=None
    )

    race_fig.update_traces(marker_color=race_colors, textposition='outside')
    race_fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=20, r=60, t=40, b=20),
    )
    
    state_max = state_df["Count"].max() if not state_df.empty else 1
    state_df["Opacity"] = 0.4 + (state_df["Count"] / state_max) * 0.6
    state_colors = [f'rgba(41, 128, 185, {opacity})' for opacity in state_df["Opacity"]]
    
    state_fig = px.bar(
        state_df, 
        y="State", 
        x="Count", 
        orientation='h', 
        title="Geographic Distribution: Top 5 States by Employee Count", 
        text="Count",
        template=None
    )
    
    state_fig.update_traces(marker_color=state_colors, textposition='outside')
    state_fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=20, r=60, t=40, b=20),
    )
    
    return race_fig, state_fig

#=============================================================================
# PAGE NAVIGATION CALLBACK
#=============================================================================
@app.callback(
    [Output('page-content', 'children'),
     Output('tab-1', 'active'),
     Output('tab-2', 'active'),
     Output('tab-3', 'active')],
    [Input('url', 'pathname')]
)
def display_page(pathname):
    if pathname == "/demographics":
        return demographics_layout, False, True, False 
    elif pathname == "/salary":
        return salary_layout, False, False, True
    return summary_layout, True, False, False

#=============================================================================
# RACE PIE CHART CALLBACK
#=============================================================================
@app.callback(
    [Output('race-pie-chart', 'figure'),
     Output('male-btn', 'color'), Output('male-btn', 'outline'),
     Output('female-btn', 'color'), Output('female-btn', 'outline')],
    [Input('male-btn', 'n_clicks'), Input('female-btn', 'n_clicks')]
)
def update_pie_chart(male_clicks, female_clicks):
    male_selected = (male_clicks or 0) % 2 == 1
    female_selected = (female_clicks or 0) % 2 == 1
    
    if not male_selected and not female_selected:
        selected_genders = ["Male", "Female"]
    else:
        selected_genders = []
        if male_selected:
            selected_genders.append("Male")
        if female_selected:
            selected_genders.append("Female")
    
    filtered = df[df["Gender"].isin(selected_genders)]
    
    fig = px.pie(
        filtered, 
        names="RaceDesc", 
        title="Racial Diversity Distribution by Employee Gender", 
        hole=0.4,
        color_discrete_sequence=px.colors.qualitative.Safe
    )
    fig.update_traces(textinfo='percent+label')
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        height=250,
        margin=dict(t=30, b=10, l=10, r=10)
    )
    
    male_color = "primary"
    female_color = "danger"
    
    male_outline = not male_selected
    female_outline = not female_selected
    
    return fig, male_color, male_outline, female_color, female_outline

#=============================================================================
# AGE EMPLOYMENT BAR CHART CALLBACK
#=============================================================================
@app.callback(
    Output('age-employment-bar', 'figure'),
    [Input('male-btn', 'n_clicks'), Input('female-btn', 'n_clicks')]
)
def update_age_employment_bar(male_clicks, female_clicks):
    male_selected = (male_clicks or 0) % 2 == 1
    female_selected = (female_clicks or 0) % 2 == 1
    
    if not male_selected and not female_selected:
        selected_genders = ["Male", "Female"]
    else:
        selected_genders = []
        if male_selected:
            selected_genders.append("Male")
        if female_selected:
            selected_genders.append("Female")
    
    filtered = df[df["Gender"].isin(selected_genders)]
    age_emp_df = filtered.groupby(["Age Group", "Employment Status"], observed=False).size().reset_index(name="Count")
    
    fig = px.bar(
        age_emp_df, 
        x="Age Group", 
        y="Count", 
        color="Employment Status", 
        title="Employment Status Distribution Across Age Group Segments", 
        barmode="overlay", 
        text=age_emp_df["Count"]
    )
    
    fig.update_traces(textposition='inside')
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        height=250,
        margin=dict(t=30, b=10, l=10, r=10)
    )
    
    return fig

#=============================================================================
# ABSENCES GRAPH CALLBACK
#=============================================================================
@app.callback(
    Output('figure_salary_performance', 'figure'),
    [Input('dropdown_department', 'value'),
     Input('slider_absences', 'value')]
)
def update_absences_graph(dept, absences):
    filtered = df[df['Absences'] <= absences]
    
    if dept != 'All':
        filtered = filtered[filtered['Department'] == dept]
    
    fig = px.box(
        filtered, 
        x='Department', 
        y='Absences', 
        color='Sex', 
        title='Employee Absenteeism: Boxplot by Department and Gender'
    )
    
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        height=300,
        margin=dict(t=30, b=30, l=50, r=30)
    )
    
    return fig

#=============================================================================
# SALARY BOX PLOT CALLBACK
#=============================================================================
@app.callback(
    Output('unfiltered_salary_graph', 'figure'),
    Input('selected-year', 'children')
)
def update_salary_box(year_str):
    year = int(year_str)
    
    filtered = df[df['Year'] == year]
    
    fig = px.box(
        filtered, 
        x='Department', 
        y='Salary', 
        color='Sex', 
        title='Salary Distribution by Department and Gender',
        color_discrete_sequence=px.colors.qualitative.Safe
    )
    
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
    )
    
    return fig

#=============================================================================
# AVG SALARY BY PERFORMANCE CALLBACK
#=============================================================================
@app.callback(
    Output('avg_salary_by_performance', 'figure'),
    Input('performance_filter', 'value')
)
def update_avg_salary_chart(performance):
    
    salary_by_marital = df.groupby(['MaritalDesc', 'PerformanceScore'], as_index=False).agg({'Salary': 'mean'})
    
    if performance != "All":
        filtered = salary_by_marital[salary_by_marital['PerformanceScore'] == performance]
    else:
        filtered = salary_by_marital
    
    
    fig = px.bar(
        filtered, 
        x='MaritalDesc', 
        y='Salary', 
        color='MaritalDesc', 
        title='Compensation Trends: Average Salary by Marital Status', 
        barmode='stack',
        color_discrete_sequence=px.colors.qualitative.Safe
    )
    
    
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        xaxis_title="Marital"
    )
    
    return fig

#=============================================================================
# RUN APP
#=============================================================================
if __name__ == '__main__':
    port = 8251
    url = f"http://127.0.0.1:{port}/"
    webbrowser.open_new_tab(url)
    app.run_server(port=port, debug=True)